In [32]:
import spacy
import json
import os
from spacy.pipeline import EntityRuler

text= "Martha, a senior, moved to Spain where she will be playing basketball until June 2022 or until she can't play any longer."

nlp = spacy.load("en_core_web_sm")

ruler = nlp.add_pipe("entity_ruler")

patterns = [{"label": "SPORT", "pattern" : "basketball"}]

ruler.add_patterns(patterns)
nlp.to_disk("sports_ner")

nlp2=spacy.load("sports_ner")
doc = nlp2(text)

for ent in doc.ents:
    print(ent.text, ent.label)


            
            

Martha 380
Spain 384
basketball 5520325248479825761
June 2022 391


In [28]:
import spacy
import json
import os
from spacy.pipeline import EntityRuler

 

current_dir = os.getcwd()
print("Current Working Directory:", current_dir)

# format of Spacy yTraining Data
# TRAIN_DATA = [(text, {"entities": [(start,end,label)]})]

def save_data(file,data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data,f, indent=4)

def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)  

data = load_data("ReferenceFiles/syslog_entities.json")
print(data)



def test_model(model, text):
    doc = nlp(text)
    results = []
    entities = []
    for ent in doc.ents:
        #results.append((ent.start_char,ent.end_char, ent.label))
        something = ((ent.start_char,ent.end_char, ent.label))
        print(something)
        
    return (results)  


def create_training_data(file,type):
    data = load_data(file)
    patterns = []
    for item in data:
        pattern = {"label" : type,
                   "pattern": item } # this is what spacy expects
        patterns.append(pattern)
    return patterns    
        
        
def generate_rules(nlp,patterns):
    entity_ruler = EntityRuler(nlp, overwrite_ents=True)
    nlp.add_pipe("ner", name="syslog_ner")
    entity_ruler.add_patterns(patterns)
    nlp.to_disk("syslog_ner")

nlp = spacy.blank("en")
#nlp = spacy.load("en_core_web_sm")            
patterns = create_training_data("ReferenceFiles/syslog_entities.json","COMPONENT") 
generate_rules(nlp,patterns)  


print(nlp.pipe_names)  

    

Current Working Directory: /home/johnos/PythonForDigitalHumanities
['WorkspaceStatus', 'Workspace_Management', 'bb_kvm_client', 'bb_rdp_client', '[OSD]']
['syslog_ner']


In [27]:
import spacy
import json
import os
from spacy.pipeline import EntityRuler


# reference https://www.youtube.com/watch?v=16Ujcah_-h0
def save_data(file, data):
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def test_model(model, text):
    doc = model(text)
    results = []
    for ent in doc.ents:
        results.append((ent.start_char, ent.end_char, ent.label_))
    return results

def create_training_data(file, type):
    data = load_data(file)
    patterns = []
    for item in data:
        pattern = {"label": type, "pattern": item} # this is what spacy expects
        patterns.append(pattern)
    return patterns

def generate_rules(nlp, patterns):
    entity_ruler = EntityRuler(nlp, overwrite_ents=True)
    entity_ruler.add_patterns(patterns)
    nlp.add_pipe("entity_ruler")
nlp = spacy.blank("en")
#nlp = spacy.load("en_core_web_sm") # Load the base English model
patterns = create_training_data("ReferenceFiles/syslog_entities.json", "COMPONENT")
generate_rules(nlp, patterns)

print(nlp.pipe_names)
nlp = spacy.load("syslog_ner")
with open("ReferenceFiles/syslog", "r") as f:
    text = f.read()
test_model(nlp, text)


['entity_ruler']


ValueError: [E109] Component 'syslog_ner' could not be run. Did you forget to call `initialize()`?

In [1]:
import spacy
from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")
entity_ruler = EntityRuler(nlp)
nlp.add_pipe("entity_ruler", before="ner")
ruler = nlp.get_pipe('entity_ruler')

skills = [
    {'label': 'SKILL', 'pattern': [{"lower": "python"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "sql"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "mysql"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "pandas"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "spacy"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "scikit-learn"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "scikit"}, {"lower": "learn"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "sklearn"}]},
    {'label': 'SKILL', 'pattern': [{"lower": "tensor"}, {"lower": "flow"}]},
]

ruler.add_patterns(skills)

text = """
We are looking for a data scientist with knowledge of Python and MySQL. 
The role will involve working with Pandas, scikit-learn, and Spacy.
Knowledge of Tensor Flow would be advantageous.
"""

doc = nlp(text)

entities = [(ent.text, ent.label_) for ent in doc.ents]
print(entities)
print(nlp.pipe_names)



[('Python', 'SKILL'), ('MySQL', 'SKILL'), ('Pandas', 'SKILL'), ('Spacy', 'SKILL'), ('Tensor Flow', 'SKILL')]
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'entity_ruler', 'ner']


In [5]:
import spacy

# You can generate a base config here: https://spacy.io/usage/training#config
nlp = spacy.blank("en") # a blank model
print(nlp.pipe_names)
nlp.add_pipe("ner") # just the ner model
print(nlp.pipe_names)
nlp = spacy.load("en_core_web_sm") #the standard small english model
print(nlp.pipe_names)


[]
['ner']
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("ner", name="syslog_ner", before="lemmatizer") # we can position our pipe before or after other components
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'syslog_ner', 'lemmatizer', 'ner']
